In [0]:
import argparse
import time
import torch
import torch.nn
from torch.autograd import Variable
import torch.nn as nn
from lm import repackage_hidden, LM_LSTM
import numpy as np
import reader

Ниже - обучение базовой модели из статьи: bigLSTM(small PTB)
Обученная модель сохранена в файле lm_model.pt  
Поэтому это можно скипнуть

In [0]:
criterion = nn.CrossEntropyLoss()
data = 'data'
hidden_size = 200
num_steps = 35
num_layers = 2
batch_size = 20
num_epochs = 13 
dp_keep_prob = 0.35
inital_lr = 20.0
save = 'lm_model.pt'

In [0]:
def repackage_hidden(h):
  """Wraps hidden states in new Variables, to detach them from their history."""
  

  if type(h) == torch.Tensor:
    return Variable(h.data)
  else:
    d = tuple(repackage_hidden(v) for v in h)
    return d

In [0]:
def run_epoch(model, data, is_train=False, lr=1.0):
    """Runs the model on the given data."""
    if is_train:
        model.train()
    else:
        model.eval()
    epoch_size = ((len(data) // model.batch_size) - 1) // model.num_steps
    start_time = time.time()
    hidden = model.init_hidden()
    hidden[0].requires_grad=True
    hidden[1].requires_grad=True
    costs = 0.0
    iters = 0
    for step, (x, y) in enumerate(reader.ptb_iterator(data, model.batch_size, model.num_steps)):
        inputs =torch.from_numpy(x.astype(np.int64)).transpose(0, 1).contiguous().cuda()
        model.zero_grad()
        hidden = repackage_hidden(hidden)
        outputs, hidden = model(inputs, hidden)
        targets = torch.from_numpy(y.astype(np.int64)).transpose(0, 1).contiguous().cuda()
        tt = torch.squeeze(targets.view(-1, model.batch_size * model.num_steps))

        loss = criterion(outputs.view(-1, model.vocab_size), tt)
        #print( loss.data.item() , model.num_steps)
        costs += loss.data.item() * model.num_steps
        iters += model.num_steps

        if is_train:
            loss.backward()
            torch.nn.utils.clip_grad_norm(model.parameters(), 0.25)
            for p in model.parameters():
                p.data.add_(-lr, p.grad.data)
            if step % (epoch_size // 10) == 10:
                print("{} perplexity: {:8.2f} speed: {} wps".format(step * 1.0 / epoch_size, np.exp(costs / iters),
                                                       iters * model.batch_size / (time.time() - start_time)))
    return np.exp(costs / iters)


In [0]:
#for google collab
!mkdir /content/data

In [8]:

raw_data = reader.ptb_raw_data(data_path=data)
train_data, valid_data, test_data, word_to_id, id_2_word = raw_data
vocab_size = len(word_to_id)
print('Vocabluary size: {}'.format(vocab_size))
model = LM_LSTM(embedding_dim=hidden_size, num_steps=num_steps, batch_size=batch_size,
                vocab_size=vocab_size, num_layers=num_layers, dp_keep_prob=dp_keep_prob)
model.cuda()
lr = inital_lr
# decay factor for learning rate
lr_decay_base = 1 / 1.15
# we will not touch lr for the first m_flat_lr epochs
m_flat_lr = 14.0

print("########## Training ##########################")
for epoch in range(num_epochs):
    lr_decay = lr_decay_base ** max(epoch - m_flat_lr, 0)
    lr = lr * lr_decay # decay lr if it is time
    train_p = run_epoch(model, train_data, True, lr)
    print('Train perplexity at epoch {}: {:8.2f}'.format(epoch, train_p))
    print('Validation perplexity at epoch {}: {:8.2f}'.format(epoch, run_epoch(model, valid_data)))
print("########## Testing ##########################")
model.batch_size = 1 # to make sure we process all the data
print('Test Perplexity: {:8.2f}'.format(run_epoch(model, test_data)))
with open(save, 'wb') as f:
    torch.save(model, f)
print("########## Done! ##########################")

Vocabluary size: 10000
########## Training ##########################


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


0.007535795026375283 perplexity:  4893.80 speed: 15333.384672284516 wps
0.10700828937452901 perplexity:  1129.30 speed: 27170.95643940926 wps
0.20648078372268275 perplexity:   894.38 speed: 28152.11265621626 wps
0.3059532780708365 perplexity:   763.39 speed: 28525.145137913034 wps
0.4054257724189902 perplexity:   687.17 speed: 28735.60676872225 wps
0.5048982667671439 perplexity:   633.69 speed: 28844.897095030483 wps
0.6043707611152976 perplexity:   584.44 speed: 28917.652977779464 wps
0.7038432554634514 perplexity:   547.92 speed: 28967.146064885612 wps
0.8033157498116051 perplexity:   519.32 speed: 29012.09219877926 wps
0.9027882441597589 perplexity:   493.47 speed: 29041.501072367202 wps
Train perplexity at epoch 0:   473.16
Validation perplexity at epoch 0:   281.79
0.007535795026375283 perplexity:   371.09 speed: 23262.19686203725 wps
0.10700828937452901 perplexity:   300.43 speed: 28737.658264237896 wps
0.20648078372268275 perplexity:   309.71 speed: 29016.716740110794 wps
0.3059

Загрузка обученной модели

In [70]:
raw_data = reader.ptb_raw_data(data_path=data)
train_data, valid_data, test_data, word_to_id, id_2_word = raw_data
# vocab_size = len(word_to_id)
# print('Vocabluary size: {}'.format(vocab_size))
model =  torch.load('./lm_model.pt')
model.cuda()
model.eval()

LM_LSTM(
  (dropout): Dropout(p=0.65)
  (word_embeddings): Embedding(10000, 200)
  (lstm): LSTM(200, 200, num_layers=2, dropout=0.65)
  (sm_fc): Linear(in_features=200, out_features=10000, bias=True)
)

In [0]:
freq = {key:0 for key in id_2_word.keys()}
for j in train_data:
    freq[j] +=1

In [0]:
from math import sqrt
freq_sqr = np.array([sqrt(q) for key, q in freq.items()])

In [0]:
def weighted_svd(A, freq_sqr, rank_to_be): 
    Q = torch.diag(torch.from_numpy(freq_sqr).cuda().type(torch.cuda.FloatTensor))
    QA = torch.matmul(Q, A)
    U_, S_, V_ = torch.svd(QA)
    #print(Q.shape, V_.shape, S_.shape)
    #print(torch.matmul(torch.inverse(Q),U_).shape)
    rank_to_be = min(torch.matrix_rank(A), rank_to_be)
    S = torch.cat((S_[:rank_to_be], torch.zeros_like(S_[rank_to_be:])))
    U = torch.matmul(torch.inverse(Q),U_) * S
    return U, V_

In [0]:
import copy
w=copy.deepcopy(model.word_embeddings.weight.data)
# del model
#del raw_data ,train_data, valid_data, test_data, word_to_id, id_2_word 
# torch.cuda.empty_cache()

In [0]:
del U, V
torch.cuda.empty_cache()

VANILA SVD 

In [120]:
u, s, v = torch.svd(w)
ns = torch.cat((s[:40], torch.zeros_like(s[40:])))
new_weight = torch.matmul(u*ns, v.t()) 
model.state_dict()['word_embeddings.weight'].data.copy_(new_weight)
print('Test Perplexity: {:8.2f}'.format(run_epoch(model, test_data)))

Test Perplexity:   149.09


Weighted SVD

In [121]:
U , V = weighted_svd(w, freq_sqr,rank_to_be = 40)
new_weight = torch.matmul(U, V.t()) 
model.state_dict()['word_embeddings.weight'].data.copy_(new_weight)
print('Test Perplexity: {:8.2f}'.format(run_epoch(model, test_data)))

Test Perplexity:   150.65


In [122]:
#rank_to_be = 40
number_of_clusters = 5 
rank_to_be_on_each_cluster = 40
clusters = list(torch.split(w, int(w.shape[0] / number_of_clusters)))
freq_cls = np.array_split(freq_sqr, number_of_clusters)
V = {}
U = {}
for p in range(number_of_clusters): 
    U[p], V[p]  = weighted_svd(clusters[p],freq_cls[p], rank_to_be_on_each_cluster)  
new_weight = torch.cat([torch.matmul(U[p], V[p].t()) for p in range(number_of_clusters)])
model.state_dict()['word_embeddings.weight'].data.copy_(new_weight)
print('Test Perplexity: {:8.2f}'.format(run_epoch(model, test_data)))

Test Perplexity:   150.51


In [0]:
def find_ranks(freq_cls, rank_min):
        ranks = [sum(i)/len(i) for i in freq_cls] 
        ranks /= ranks[-1] 
        ranks *=rank_min 
        ranks = {i : int(ranks[i]) for i in range(len(ranks))}
        return ranks

In [134]:
rank_min = 40 
number_of_clusters = 5 
clusters = list(torch.split(w, int(w.shape[0] / number_of_clusters)))
freq_cls = np.array_split(freq_sqr, number_of_clusters)
V = {}
U = {}
cluster_ranks = find_ranks(freq_cls, rank_min)
 
for p in range(number_of_clusters): 
    U[p], V[p]  = weighted_svd(clusters[p],freq_cls[p], int(cluster_ranks[p]))  
    
new_weight = torch.cat([torch.matmul(U[p], V[p].t()) for p in range(number_of_clusters)])
model.state_dict()['word_embeddings.weight'].data.copy_(new_weight)
print('Test Perplexity: {:8.2f}'.format(run_epoch(model, test_data)))


Test Perplexity:   146.91


In [0]:
np.save( 'new_weights.npy', new_weights)
np.save( 'ranks.npy', np.array(ranks))
np.save( 'word_in_cls.npy', np.array(word_in_cls))

without rank recalculation every time

In [0]:

def group_reduce(A, c, r, t_max, m_min):
    #init clusters
    def find_num(i):
        for j in range(c):
            i -= clusters_len[j]
            if i < 0:
                return j, clusters_len[j] + i
        
        
        return 0, 0
    
    
    def find_ranks(freq_cls, rank_min):
        ranks = [sum(i)/len(i) for i in freq_cls] 
        ranks /= ranks[-1] 
        ranks *=rank_min 
        ranks = {i : int(ranks[i]) for i in range(len(ranks))}
        return ranks
    
    clusters_len = []
    clusters = list(torch.split(A, int(A.shape[0] / c)))
    for cluster in clusters:
        clusters_len.append(cluster.shape[0])
    words_in_cls = np.array_split(list(id_2_word.keys()),c)
    freq_cls = np.array_split(freq_sqr, c)
    ranks = find_ranks(freq_cls, r)
    V = {}
    U = {}
    V_Vt = []

    for p in range(c): 
        U[p], V[p]  = weighted_svd(clusters[p],freq_cls[p], ranks[p])
        ranks[p] = torch.matrix_rank(torch.matmul(U[p], V[p].t()))
    for t in range(t_max):
        M = {}
        Err = []
        for p in range(c):
            error_p = torch.sqrt((A - torch.matmul(A, torch.matmul(V[p], V[p].t()))).pow(2).sum(1))
            Err.append(error_p)
        Err = torch.stack(Err, dim = 0)
        e, g = torch.min(Err, dim = 0)
        del Err
        del error_p
        torch.cuda.empty_cache()
        
        for i in range(A.shape[0]):
            if g[i] !=  find_num(i)[0]:
                M[i] = (e[i], g[i])
                
        m = 0.1*len(M)
        t = 0
        is_changed = [False]*c
        for key, value in sorted(M.items(), key=lambda kv: kv[1]):
            
            words_in_cls[value[1]] = np.append(words_in_cls[value[1]], words_in_cls[find_num(key)[0]][find_num(key)[1]])
            words_in_cls[find_num(key)[0]] = np.delete(words_in_cls[find_num(key)[0]], find_num(key)[1])
            
            freq_cls[value[1]] = np.append(freq_cls[value[1]], freq_cls[find_num(key)[0]][find_num(key)[1]])
            freq_cls[find_num(key)[0]] = np.delete(freq_cls[find_num(key)[0]], find_num(key)[1])

            clusters[value[1]] = torch.cat((clusters[value[1]],torch.unsqueeze(clusters[find_num(key)[0]][find_num(key)[1]],dim = 0)))
            clusters[find_num(key)[0]] = torch.cat((clusters[find_num(key)[0]][ :find_num(key)[1]], 
                                                     clusters[find_num(key)[0]][find_num(key)[1] + 1: ]), dim = 0)

            clusters_len[find_num(key)[0]] -= 1
            clusters_len[value[1]] += 1
            is_changed[value[1]] = True
            is_changed[find_num(key)[0]] = True
#             print("Hooray")
            t += 1
            if m == t:
                break;
        if m < m_min:
            return [torch.matmul(U[p], V[p].t()) for p in range(c)] ,ranks, words_in_cls
#        ranks= find_ranks(freq_cls, r)
        for p in range(c):
            if is_changed[p]:
                
                U[p], V[p]  = weighted_svd(clusters[p],freq_cls[p], ranks[p] )
                #ranks[p] = torch.matrix_rank(torch.matmul(U[p], V[p].t()))
                
    return [torch.matmul(U[p], V[p].t()) for p in range(c)], ranks, words_in_cls

In [133]:
new_weights , ranks , word_in_cl = group_reduce(w, c = 5, r = 40, t_max = 100, m_min = 10)
nw = torch.empty_like(w)
for i in range(10000):
    idx =np.where(np.concatenate(word_in_cls)==i)
    nw[i]=torch.cat(new_weights)[idx[0]]
model.state_dict()['word_embeddings.weight'].data.copy_(nw)
print('Test Perplexity: {:8.2f}'.format(run_epoch(model, test_data)))

(array([8014]),)
Test Perplexity:   148.34
